<a href="https://colab.research.google.com/github/hashiraliahmed/Portfolio/blob/main/Stat4188_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [1]:
print("Hello World")

Hello World


In [2]:
print("!")

!


In [4]:
print("testing GitHub connection")

testing GitHub connection
